In [14]:
import transformers
from transformers import (
    AutoConfig, BertConfig, BertTokenizer, BertForRetrieval, BertForSequenceClassification)
import torch
import numpy as np

In [162]:
#ckpt_dir = "/home/transformers-public/output_dirs/sarah-test6/checkpoint-6000" # Good!
ckpt_dir = "/home/transformers-public/output_dirs/sarah-test7/checkpoint-4000" # Good!

# ckpt_dir = "../output_dirs/classification_test/checkpoint-4500"

In [156]:
config = AutoConfig.from_pretrained(ckpt_dir)
tokenizer = BertTokenizer.from_pretrained(
    ckpt_dir,
    do_lower_case=True,
)
model = BertForSequenceClassification.from_pretrained(
    ckpt_dir,
    from_tf=False,
    config=config,
)

In [157]:
use_gpu = True
if use_gpu:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cpu")
print(device)

cuda


In [158]:
model = model.to(device)

In [149]:
max_seq_length=128
test_input = tokenizer.batch_encode_plus(
    [("test input", None)],
    max_length=max_seq_length,
    pad_to_max_length=True,
)

In [124]:
model_inp = dict(test_input)
model_inp['labels'] = [0]
# print(model_inp)
model_inp = {k: torch.tensor(v).to(device) for k, v in model_inp.items()}
model_out = model(**model_inp)

In [125]:
len(model_out)

3

In [35]:
print([x.shape for x in model_out])  # loss, logits, pooled_output (i.e. encoding)

[torch.Size([]), torch.Size([1, 2]), torch.Size([1, 768])]


In [36]:
predicted_logits = model_out[1].detach().cpu().numpy()
predicted_class = np.argmax(predicted_logits)

In [37]:
print(predicted_logits)

[[ 0.43291587 -0.18428318]]


In [26]:
print(predicted_class)

1


# Run on Doordash Data

In [73]:
import csv

dataset = []
with open("doordash_healthy.csv", newline='',encoding = "utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        dataset.append({
            'title': row['title'],
            'description': row['description'],
        })
        


In [159]:
labels = ['healthy', 'unhealthy']

counts = [0, 0]
for p in dataset:
    text = p['title'].lower() + ' ' + p['description'].lower()
    test_input = tokenizer.batch_encode_plus(
        [(text, None)],
        max_length=max_seq_length,
        pad_to_max_length=True,
    )
    
    model_inp = dict(test_input)
    model_inp['labels'] = [0]
    model_inp = {k: torch.tensor(v).to(device) for k, v in model_inp.items()}
    model_out = model(**model_inp)

    predicted_logits = model_out[1].detach().cpu().numpy()
    predicted_class = np.argmax(predicted_logits)
    
    if labels[predicted_class] == 'unhealthy':
        print(text)
    
    #print(labels[predicted_class], predicted_logits)
    counts[predicted_class] += 1
    #print()


mcveggie burger (2010 kj.)
ensalada de papa potato salad.
fresh guacamole dip fresh and buttery avocados, cilantro, jalapeños, onions, lime juice and special spices.
vegetarian mapo tofu hot and spicy.
fusilli alfredo vegetarian. with parmesan cream sauce. 750 cal.
pita & dip fresh made pita chips comes with a choice of humus
colombian empanadas vegetarian, cheese. colombian empanadas.
garlic mashed potatoes add 
szechuan tofu lettuce wraps (v) crispy szechuan tofu, peanuts, wontons, korean chili sauce, spicy yogurt. note: spicy yogurt contains egg and dairy, wontons contain egg.

dietary info: vegetarian
vegetarian b one bean tostada, one cheese enchilada,  served with rice and refried beans.
snow peas with water chestnut 
black bean patty - super 910 cal.
peaches ‘n silk gluten-friendly. vegetarian. peaches, strawberries & banana.
black bean burrito a large flour tortilla filled with our homemade black beans, mexican rice, salsa fresca and cheeses. served with a cafe salad.
3. sautée

In [160]:
counts

[430, 72]

# Sanity Check 

In [161]:
labels = ['healthy', 'unhealthy']

samples = [
    "chocolate ice cream",
    "big mac", 
    "coca cola", 
    "fried oreos", 
    "salad", 
    "ramen",
    "beer", 
    "gyro sandwich",
]

for text in samples:

    test_input = tokenizer.batch_encode_plus(
        [(text, None)],
        max_length=max_seq_length,
        pad_to_max_length=True,
    )
    
    model_inp = dict(test_input)
    model_inp['labels'] = [0]
    model_inp = {k: torch.tensor(v).to(device) for k, v in model_inp.items()}
    model_out = model(**model_inp)

    predicted_logits = model_out[1].detach().cpu().numpy()
    predicted_class = np.argmax(predicted_logits)
    
    print(text)
    print(labels[predicted_class], predicted_logits)
    print()

chocolate ice cream
unhealthy [[-0.19751902  0.04265981]]

big mac
unhealthy [[-3.4547448  2.9443698]]

coca cola
unhealthy [[-0.34153998  0.14552867]]

fried oreos
healthy [[ 0.16661696 -0.10494222]]

salad
healthy [[ 0.3661355 -0.3513371]]

ramen
healthy [[ 0.5504143 -0.380982 ]]

beer
healthy [[ 0.1510079  -0.09273873]]

gyro sandwich
healthy [[ 0.21280293 -0.07180934]]

